<a href="https://colab.research.google.com/github/husaintheking/CMLS1/blob/main/CMLS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lib import

In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras

#https://medium.com/@sdoshi579/classification-of-music-into-different-genres-using-keras-82ab5339efe0

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd "/content/drive/MyDrive/Input"
%ls

/content/drive/.shortcut-targets-by-id/10h63gkOCyEstkEYfz8l4GXxjT18LPixT/Input
Distortion/  NoFX/  Tremolo/


In [ ]:
# creare un array con i nomi delle cartelle folders = [Distortion, NoFX, Tremolo]
#ciclo for x in folders.length analizzate tutte le cartelle (es folders[0] = "Distortion" mi muovo in questa cartella e analizzo ogni elemento, folders[1], folders[2]); x parte da 0

#consiglio: mettete tutti le analisi in un file json, ve lo salvate e poi analizzate i valori all'interno senza dover riprocessare tutto

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
#files path (DA AGGIUSTARE E PROVARE)
curr_dir =  "/content/drive/My Drive/path/to/files/"
%cd "$curr_dir"

In [ ]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
effects = 'NoFX Tremolo Distortion'.split()
for fx in effects:
    for filename in os.listdir(f'/content/drive/MyDrive/mfcctest/DatasetTest/{fx}'):
        songname = f'/content/drive/MyDrive/mfcctest/DatasetTest/{fx}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {fx}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
data = pd.read_csv('data.csv')
data.head()

In [ ]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
data.head()

## **Feature definition**

### MFCC

THC

Temporal centroid

Local Maxima number

In [ ]:
def find_maxima(signal):

  analytic_signal = hilbert(signal)   #hilbert transformation
  amp_env = np.abs(analytic_signal)   

  # for local maxima - positions
  max = len(argrelextrema(amp_env, np.greater))

  # for local minima - positions
  #min = argrelextrema(amp_env, np.less)

return max

### Compute training feature 

In [ ]:
classes = ['blues', 'disco', 'metal']
n_mfcc = 13
dict_train_features = {'blues': [], 'metal': [], 'disco': []}

for c in classes:
    train_root = 'Input/{}/training/'.format(c)
    class_train_files = [f for f in os.listdir(train_root) if f.endswith('.wav')]
    n_train_samples = len(class_train_files)
    
    train_features = np.zeros((n_train_samples, n_mfcc))
    for index, f in enumerate(class_train_files):
        audio, fs = librosa.load(os.path.join(train_root, f), sr=None)
        mfcc = compute_mfcc(audio, fs, n_mfcc)
        train_features[index, :] = np.mean(mfcc, axis=1)
        
    dict_train_features[c] = train_features

### Compute testing feature

# **SVM**

In [ ]:
class_0 = 'blues'
class_1 = 'disco'
class_2 = 'metal'

X_train_0 = dict_train_features[class_0]
X_train_1 = dict_train_features[class_1]
X_train_2 = dict_train_features[class_2]

y_train_0 = np.zeros((X_train_0.shape[0],))
y_train_1 = np.ones((X_train_1.shape[0],))
y_train_2 = np.ones((X_train_2.shape[0],))*2

#y_train = np.concatenate((y_train_class_0, y_train_class_1, y_train_class_1), axis=0)

In [ ]:
X_test_0 = dict_test_features[class_0]
X_test_1 = dict_test_features[class_1]
X_test_2 = dict_test_features[class_2]


y_test_0 = np.zeros((X_test_0.shape[0],))
y_test_1 = np.ones((X_test_1.shape[0],))
y_test_2 = np.ones((X_test_2.shape[0],))*2

y_test_mc = np.concatenate((y_test_0, y_test_1, y_test_2), axis=0)

### Normalization

In [ ]:
feat_max = np.max(np.concatenate((X_train_0, X_train_1, X_train_2), axis=0), axis=0)
feat_min = np.min(np.concatenate((X_train_0, X_train_1, X_train_2), axis=0), axis=0)

X_train_0_normalized = (X_train_0 - feat_min) / (feat_max - feat_min)
X_train_1_normalized = (X_train_1 - feat_min) / (feat_max - feat_min)
X_train_2_normalized = (X_train_2 - feat_min) / (feat_max - feat_min)

X_test_0_normalized = (X_test_0 - feat_min) / (feat_max - feat_min)
X_test_1_normalized = (X_test_1 - feat_min) / (feat_max - feat_min)
X_test_2_normalized = (X_test_2 - feat_min) / (feat_max - feat_min)

X_test_mc_normalized = np.concatenate((X_test_0_normalized, X_test_1_normalized, X_test_2_normalized), axis=0)

### Models definition

In [ ]:
SVM_parameters={
    'C': 1,
    'kernel': 'rbf',
}

clf_01 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_02 = sklearn.svm.SVC(**SVM_parameters, probability=True)
clf_12 = sklearn.svm.SVC(**SVM_parameters, probability=True)



### Training models

In [ ]:
clf_01.fit(np.concatenate((X_train_0_normalized, X_train_1_normalized), axis=0), 
           np.concatenate((y_train_0, y_train_1), axis=0))
           
clf_02.fit(np.concatenate((X_train_0_normalized, X_train_2_normalized), axis=0), 
           np.concatenate((y_train_0, y_train_2), axis=0))

clf_12.fit(np.concatenate((X_train_1_normalized, X_train_2_normalized), axis=0), 
           np.concatenate((y_train_1, y_train_2), axis=0))